In [6]:
%matplotlib widget

In [7]:

import pandas as pd

import matplotlib.pyplot as plt
import pandas_ta as ta

## Load parquet

In [135]:
df = pd.read_parquet("data.parquet")

In [136]:
df.groupby("name").count()

,id,figi,ts,open_price,close_price,high_price,low_price,volume
name,,,,,,,,
AT&T,101134,101134,101134,101134,101134,101134,101134,101134
Apple,119367,119367,119367,119367,119367,119367,119367,119367
JPMorgan,85536,85536,85536,85536,85536,85536,85536,85536


## Apply processing
Strategy can be applied only on by-name group

In [137]:
def concat_df_with_col(df, column):
    return pd.concat([df, column], axis=1)

In [138]:
def calc_metrics(df, 
    RSI_oversold_threshold = 30,
    RSI_overbouht_threshold = 70,
    MFI_oversold_threshold = 20,
    MFI_overbouht_threshold = 80):
    # Add MACD
    df = concat_df_with_col(df, df.groupby("name").apply(lambda x: x.ta.macd()).reset_index(level=0, drop=True))

    # Add RSI
    df = concat_df_with_col(df, df.groupby("name").apply(lambda x: x.ta.rsi()).reset_index(level=0, drop=True))

    # Add MFI
    df = concat_df_with_col(df, df.groupby("name").apply(lambda x: x.ta.mfi()).reset_index(level=0, drop=True))
    
    # Add RSI oversold and overbought
    df["RSI_oversold"] = df["RSI_14"] <= RSI_oversold_threshold
    df["RSI_overbought"] = df["RSI_14"] >= RSI_overbouht_threshold

    # Add MFI oversold and overbought
    df["MFI_oversold"] = df["MFI_14"] <= MFI_oversold_threshold
    df["MFI_overbought"] = df["MFI_14"] >= MFI_overbouht_threshold
    
    # Add MACD signal crossover and cross under
    df["MACD_diff"] = (df["MACD_12_26_9"] - df["MACDs_12_26_9"])
    df["MACD_crossover"] = (df["MACD_diff"] >= 0) & (df["MACD_diff"].shift() <= 0)
    df["MACD_crossunder"] = (df["MACD_diff"] <= 0) & (df["MACD_diff"].shift() >= 0)
    
    return df

In [139]:
df = calc_metrics(df)

## Show result

In [140]:
df.head()

,name,id,figi,ts,open_price,close_price,high_price,low_price,volume,MACD_12_26_9,...,MACDs_12_26_9,RSI_14,MFI_14,RSI_oversold,RSI_overbought,MFI_oversold,MFI_overbought,MACD_diff,MACD_crossover,MACD_crossunder
0,JPMorgan,1,BBG000DMBXR2,2018-01-23 21:45:00+00:00,114.02,114.02,114.02,114.02,6.0,NaN,...,NaN,NaN,NaN,False,False,False,False,NaN,False,False
1,JPMorgan,1,BBG000DMBXR2,2018-01-23 21:35:00+00:00,114.02,114.02,114.02,114.02,10.0,NaN,...,NaN,NaN,NaN,False,False,False,False,NaN,False,False
2,JPMorgan,1,BBG000DMBXR2,2018-01-23 20:55:00+00:00,114.20,114.23,114.26,114.16,76837.0,NaN,...,NaN,100.000000,NaN,False,True,False,False,NaN,False,False
3,JPMorgan,1,BBG000DMBXR2,2018-01-23 20:50:00+00:00,114.37,114.21,114.39,114.19,34299.0,NaN,...,NaN,90.697674,NaN,False,True,False,False,NaN,False,False
4,JPMorgan,1,BBG000DMBXR2,2018-01-23 20:45:00+00:00,114.33,114.36,114.41,114.31,27240.0,NaN,...,NaN,94.688458,NaN,False,True,False,False,NaN,False,False


## Save results

In [141]:
df.to_parquet("processed_data.parquet", compression="gzip")